In [1]:
# https://www.databricks.com/blog/efficient-fine-tuning-lora-guide-llms
import pandas as pd
from datasets import load_dataset

/home/tbiradar/calligraphy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading and Preprocessing
Convert CSV into instruction-format for finetuning 

In [2]:
similes_ds = load_dataset('csv', data_files='./data/similes.csv')
similes_df = pd.DataFrame(similes_ds['train'])

similes_df.head()

,input,output
0,Academic Advisor,"A good academic advisor is like a lighthouse, ..."
1,Academic Advisor,"An academic advisor is like a tour guide, lead..."
2,Academic Advisor,"He was like a lighthouse, guiding me through t..."
3,Academic Advisor,"My academic advisor was like a lighthouse, gui..."
4,Academic Advisor,"My academic advisor was like a lighthouse, gui..."


In [3]:
# Combine the two attributes into an instruction string
# https://textfx.withgoogle.com/
similes_df['instruction'] = 'Create a simile that illustrates this concept: ' + similes_df['input']
similes_df = similes_df[['instruction', 'output']]

similes_df.head()

,instruction,output
0,Create a simile that illustrates this concept:...,"A good academic advisor is like a lighthouse, ..."
1,Create a simile that illustrates this concept:...,"An academic advisor is like a tour guide, lead..."
2,Create a simile that illustrates this concept:...,"He was like a lighthouse, guiding me through t..."
3,Create a simile that illustrates this concept:...,"My academic advisor was like a lighthouse, gui..."
4,Create a simile that illustrates this concept:...,"My academic advisor was like a lighthouse, gui..."


In [4]:
# similes_df_sample = similes_df.sample(n=20, random_state=42) # TODO: change for full fine-tuning
similes_df_sample = similes_df

instruction_template = "<s>[INST] {} [/INST] "

similes_df_sample['prompt'] = similes_df_sample["instruction"].apply(lambda x: instruction_template.format(x))
similes_df_sample.rename(columns={'output': 'response'}, inplace=True)

similes_df_sample.head()

,instruction,response,prompt
0,Create a simile that illustrates this concept:...,"A good academic advisor is like a lighthouse, ...",<s>[INST] Create a simile that illustrates thi...
1,Create a simile that illustrates this concept:...,"An academic advisor is like a tour guide, lead...",<s>[INST] Create a simile that illustrates thi...
2,Create a simile that illustrates this concept:...,"He was like a lighthouse, guiding me through t...",<s>[INST] Create a simile that illustrates thi...
3,Create a simile that illustrates this concept:...,"My academic advisor was like a lighthouse, gui...",<s>[INST] Create a simile that illustrates thi...
4,Create a simile that illustrates this concept:...,"My academic advisor was like a lighthouse, gui...",<s>[INST] Create a simile that illustrates thi...


In [5]:
similes_df_sample['response'] = similes_df_sample['response'] + "</s>"
similes_df_sample = similes_df_sample[['prompt', 'response']]

similes_df_sample['text'] = similes_df_sample["prompt"] + similes_df_sample["response"]
similes_df_sample.drop(columns=['prompt', 'response'], inplace=True)

similes_df_sample.head()

,text
0,<s>[INST] Create a simile that illustrates thi...
1,<s>[INST] Create a simile that illustrates thi...
2,<s>[INST] Create a simile that illustrates thi...
3,<s>[INST] Create a simile that illustrates thi...
4,<s>[INST] Create a simile that illustrates thi...


In [6]:
similes_df_sample['text'][0]

'<s>[INST] Create a simile that illustrates this concept: Academic Advisor [/INST] A good academic advisor is like a lighthouse, guiding students through the stormy seas of academia to a safe harbor.</s>'

## Train LoRA on Mistral7B

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, LoftQConfig, get_peft_model, PeftModel
from datasets import Dataset
import wandb

In [8]:
wandb.init(
    project="calligraphy",
    # track hyperparameters and run metadata
    config={
        "architecture": "QLoRA",
        "dataset": "similes",
        "epochs": 10,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dc7wfsk0 (imitation-learning-experiments). Use `wandb login --relogin` to force relogin


In [9]:
# can change the quantization type using
# https://huggingface.co/blog/4bit-transformers-bitsandbytes

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
base_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map='auto')

Loading checkpoint shards: 100%|██████████| 3/3 [00:40<00:00, 13.43s/it]


In [10]:
# load existing adapter
model = PeftModel.from_pretrained(base_model, 'mistral_similes/similes_adapter')

In [11]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer_function = lambda x : tokenizer(x['text'], truncation=True, padding='max_length', max_length=1024)

dataset = Dataset.from_pandas(similes_df_sample)
tokenized_dataset = dataset.map(
    tokenizer_function,
    batched=True,
    num_proc=4
)

print(tokenized_dataset)

Map (num_proc=4): 100%|██████████| 2528/2528 [00:00<00:00, 2953.66 examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 2528
})


In [16]:
loftq_config = LoftQConfig(loftq_bits=4)
lora_config = LoraConfig(
        r=16,
        lora_alpha=8,
        lora_dropout=0.05,
        task_type='CAUSAL_LM',
        init_lora_weights='loftq',
        loftq_config=loftq_config,
        target_modules=['q_proj', 'k_proj', 'v_proj', 'out_proj']
    )

In [17]:
model = get_peft_model(model, lora_config, adapter_name='similes_adapter')

model.print_trainable_parameters()

trainable params: 9,437,184 || all params: 7,260,606,464 || trainable%: 0.1299779026282728


In [18]:
# https://github.com/rajpurkar/cs197-lec4/blob/master/demo.ipynb

training_args = TrainingArguments(
    f'mistral_similes',
    logging_dir='./logs',
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=10,
    report_to='wandb'
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    dataset_text_field='text',
    args=training_args
)

/home/tbiradar/calligraphy/env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 2528/2528 [00:00<00:00, 13190.19 examples/s]
/home/tbiradar/calligraphy/env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [19]:
trainer.train()
trainer.model.save_pretrained(f'mistral_similes')

Step,Training Loss
500,1.220800
1000,1.047400
1500,0.983400


KeyboardInterrupt: 